In [1]:
!pip install huggingface-hub -qU

DEPRECATION: nb-black 1.0.7 has a non-standard dependency specifier black>='19.3'. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nb-black or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
from datasets import load_dataset
from huggingface_hub import login, logout
from dotenv import load_dotenv
import os

In [19]:
load_dotenv()

True

In [20]:
hf_token = os.environ["HF_TOKEN"]

In [5]:
dataset_id = "glaiveai/glaive-function-calling-v2"
new_dataset_id = "lalanikarim/glaive-function-calling-v2"

dataset = load_dataset(dataset_id,split="train")
dataset

Dataset({
    features: ['chat', 'system'],
    num_rows: 112960
})

In [6]:
def clean_data(record):
    system = record["system"] 

    """
    record["system"] = system.replace("SYSTEM:","System:")
    """
    
    chat = record["chat"]
    
    chat = (chat
            #.replace("USER:","Human:")
            .replace("ASSISTANT: <functioncall>","FUNCTION:")
            #.replace("ASSISTANT:","AI:")
            #.replace("FUNCTION RESPONSE:","Response:")
            .replace("<|endoftext|>","")
           )
    
    chat = chat.split("\n\n\n")

    record["prompt"] = chat[0]
    record["response"] = "\n".join(chat[1:])
    
    return record

In [7]:
dataset = dataset.filter(lambda c: c["chat"].find("\n\n\n") > 1)
dataset = dataset.map(clean_data)
dataset

Dataset({
    features: ['chat', 'system', 'prompt', 'response'],
    num_rows: 79643
})

In [8]:
seed = 421337
test_size = 0.1
split_dataset = dataset.shuffle(seed=421337).train_test_split(test_size=test_size)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['chat', 'system', 'prompt', 'response'],
        num_rows: 71678
    })
    test: Dataset({
        features: ['chat', 'system', 'prompt', 'response'],
        num_rows: 7965
    })
})

In [21]:
login(token=hf_token, write_permission=True)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/karim/.cache/huggingface/token
Login successful


In [11]:
split_dataset.push_to_hub(new_dataset_id)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/72 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/lalanikarim/glaive-function-calling-v2/commit/6dc2413b756af851ec7c632b2044c48cf4b39841', commit_message='Upload dataset', commit_description='', oid='6dc2413b756af851ec7c632b2044c48cf4b39841', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
hf_env_vars = ["HF_TOKEN","HUGGING_FACE_HUB_TOKEN"]

for var in hf_env_vars:
    if var in os.environ:
        del os.environ[var]

In [23]:
logout()

Successfully logged out.
